In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!pip install -q --upgrade ipython==5.5.0
!pip install -q --upgrade ipykernel==4.6.0

     |████████████████████████████████| 112kB 3.5MB/s 


In [3]:
cd /content/drive/My\ Drive/textual_analysis_email/fspool/autoencoder

/content/drive/My Drive/textual_analysis_email/fspool/autoencoder


In [0]:
# cd /content/drive/My\ Drive/projects/textual_analysis_email/fspool/autoencoder

In [5]:
!pip install cardinality

  Created wheel for cardinality: filename=cardinality-0.1.1-cp36-none-any.whl size=2604 sha256=d22e00884c424cb868ec972ce52ca714f2b4c4ae398fe54e0e6964f4c6c3541e
  Stored in directory: /root/.cache/pip/wheels/6a/e6/49/f402e4a90cc81761199d852dfdc3195169af13e2e09608f053
Successfully built cardinality


In [0]:
import os
import argparse
from datetime import datetime
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.multiprocessing as mp
import numpy as np
import torch.nn.functional as F
import torch.optim as optim
import scipy.optimize
from sklearn.metrics import roc_auc_score
from tqdm import tqdm
import matplotlib.pyplot as plt
import pandas as pd

from model import *
from data_reader import load_bipartite_hypergraph,pad_zeros
from data_processor import data_process
from node2vec import *
import data
import track
from sklearn.model_selection import train_test_split

In [7]:
cd /content/drive/My\ Drive/textual_analysis_email

/content/drive/My Drive/textual_analysis_email


In [0]:
# cd /content/drive/My\ Drive/projects/textual_analysis_email

In [9]:
cd /content/drive/My\ Drive/textual_analysis_email/s2slp

/content/drive/My Drive/textual_analysis_email/s2slp


In [0]:
# cd /content/drive/My\ Drive/projects/textual_analysis_email/s2slp

In [0]:
# home_path = '/content/drive/My Drive/projects/textual_analysis_email/'

home_path = '/content/drive/My Drive/textual_analysis_email/'

# sample_path = os.path.join(home_path, 'sample_data')

data_params = {'home_path': home_path,
               'r_label_file': 'id_p_map.txt',
               'u_label_file': 'id_a_map.txt',
               'v_label_file': 'id_k_map.txt',
               'r_u_list_file': 'p_a_list_train.txt',
               'r_v_list_file': 'p_k_list_train.txt',
               'emb_pkl_file': 'nodevectors.pkl'}
# methods = [commonneigh, admic_adar, jaccard]
# method_name_map = dict(zip(methods, ['CN', 'AA', 'JC']))
num_iter = 2

In [12]:
pos_A, pos_B = load_bipartite_hypergraph(data_params)
G, obs_pos, unobs_data, V_offset = data_process(pos_A, pos_B, neg_pos_ratio = 1, unobs_ratio=0.9)

max_id=max(list(G.nodes))
max_id


embedding_map=model(G)

emb_map=[]

for i in range(0,max_id):
  emb_map.append(embedding_map.get_vector(str(i)))

emb_map.append(np.zeros(128,dtype='float'))


Walk iteration:
1 / 10
2 / 10
3 / 10
4 / 10
5 / 10
6 / 10
7 / 10
8 / 10
9 / 10
10 / 10


In [0]:
def mapping(data,max_id):


    pairs, labels = zip(*data)
    U, V = zip(*pairs)

    n_points_U = np.array([len(x) for x in U])
    n_points_V = np.array([len(x) for x in V])
    cardinality_U = max(n_points_U)
    cardinality_V = max(n_points_V)


    U = [x + [max_id]*(cardinality_U - len(x)) for x in U]
    V = [x + [max_id]*(cardinality_V - len(x)) for x in V]
    return U, V, n_points_U, n_points_V, cardinality_U, cardinality_V, labels


In [0]:
# len(train_data),len(test_data),len(unobs_data)
# import pandas as pd
# pd.DataFrame(train_data, columns = ['pair', 'label'])
# pd.DataFrame(test_data, columns = ['pair', 'label'])

In [0]:
class FSEncoder(nn.Module):
    def __init__(self, *, input_channels, output_channels, dim, **kwargs):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv1d(input_channels, dim, 1),
            nn.ReLU(inplace=True),
            nn.Conv1d(dim, dim, 1),
        )
        self.lin = nn.Sequential(
            nn.Linear(dim, dim, 1),
            nn.ReLU(inplace=True),
            nn.Linear(dim, output_channels, 1),
        )
        self.pool = FSPool(dim, 20, relaxed=kwargs.get('relaxed', True))
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x, n_points, *args):
        x = self.conv(x)
        x, perm = self.pool(x, n_points)

        x = self.lin(x)
        return x, perm

class BLP(nn.Module):
    def __init__(self, *, input_channels, output_channels, dim, **kwargs):
        super().__init__()
        self.enc_U = FSEncoder(input_channels = input_channels,
                                output_channels = output_channels,
                                set_size = kwargs['set_size_U'],
                               dim = dim,
                                **kwargs)
        self.enc_V = FSEncoder(input_channels = input_channels,
                                output_channels = output_channels,
                                set_size = kwargs['set_size_V'],
                               dim = dim,
                                **kwargs)
        self.classifier = nn.Linear(2*output_channels, 1)
    def forward(self, sample, *args):
        U, V, n_points_U, n_points_V = sample
        x_U, _ = self.enc_U(U, n_points_U)
        x_V, _ = self.enc_V(V, n_points_V)
        x = self.classifier(torch.cat([x_U, x_V], dim=1))
        return x

class EMB_LAYER(nn.Module):
    def __init__(self,word_map,padd,input_channels, output_channels, dim, set_size_U,set_size_V,**kwargs):
        super().__init__()
        kwargs['set_size_U'] = set_size_U
        kwargs['set_size_V'] = set_size_V
        

        self.embedding = nn.Embedding.from_pretrained(embeddings=word_map,freeze=False,padding_idx=padd)

        self.out_ = BLP(input_channels = input_channels,
                                output_channels = output_channels,
                               dim = dim,
                                **kwargs)
        self.c_U=set_size_U
        self.c_V=set_size_V
        self.padd=padd

    def forward(self, sample, *args):
        U, V, n_points_U, n_points_V,mask_U,mask_V = sample
        U_=self.embedding(U)
        V_=self.embedding(V)

        #print(U_.size())
        
        """mask needed"""

        

        U_=torch.cat([U_,mask_U],dim=2)
        
        V_=torch.cat([V_,mask_V],dim=2)



        sample=(torch.transpose(U_,1,2),torch.transpose(V_,1,2),n_points_U,n_points_V)
        x=self.out_(sample)
        return x

In [19]:
weight=torch.from_numpy(np.matrix(emb_map)).type(torch.FloatTensor)


weight=torch.randn((weight.size(0),weight.size(1))).type(torch.FloatTensor)
weight[weight.size(0)-1][:]=0
weight[weight.size(0)-1]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.])

In [0]:


hidden_dim = 128
latent_dim = 32


U, V, n_points_U, n_points_V, cardinality_U, cardinality_V, labels = mapping(unobs_data, max_id)
unobs_data=list(zip(U, V, n_points_U, n_points_V, labels))
train_data,test_data=train_test_split(unobs_data,test_size=0.2)
U, V, n_points_U, n_points_V, labels=zip(*train_data)

U=torch.from_numpy(np.array(U))
V=torch.from_numpy(np.array(V))



tU, tV, tn_points_U, tn_points_V, tlabels=zip(*test_data)
tU=torch.from_numpy(np.array(tU))
tV=torch.from_numpy(np.array(tV))



# tpoints_U, tpoints_V, tn_U, tn_V, tc_U, tc_V, t_labels = mapping(embedding_map,test_data)

# points_U = pad_zeros(torch.Tensor(points_U), max([c_U,tc_U]))
# points_V = pad_zeros(torch.Tensor(points_V), max([c_V,tc_V]))

# tpoints_U = pad_zeros(torch.Tensor(tpoints_U), max([c_U,tc_U]))
# tpoints_V = pad_zeros(torch.Tensor(tpoints_V), max([c_V,tc_V]))

In [0]:
U.size()

In [0]:

# net = BLP(input_channels = 129,
#           output_channels = latent_dim,
#           set_size_U = cardinality_U,
#           set_size_V = cardinality_V,
#           dim = hidden_dim,
#           skip = False,
#           relaxed = False)


net=EMB_LAYER(weight,max_id,129,
              latent_dim,hidden_dim,
              set_size_U=int(cardinality_U),
              set_size_V = int(cardinality_V),
              skip=False,relaxed=False)

optimizer = torch.optim.Adam(net.parameters(), lr=0.0001, weight_decay=1E-6)

In [0]:
# net.load_state_dict(torch.load('fs_pool_authors_keywords.model'))

In [0]:
mask_U = torch.from_numpy(np.array([[1]*n_points_U[i] + [0]*(cardinality_U-n_points_U[i]) for i in range(len(U))])).type(torch.FloatTensor).view(U.shape[0],cardinality_U,1)
mask_V= torch.from_numpy(np.array([[1]*n_points_V[i] + [0]*(cardinality_V-n_points_V[i]) for i in range(len(V))])).type(torch.FloatTensor).view(V.shape[0],cardinality_V,1)
tmask_U = torch.from_numpy(np.array([[1]*tn_points_U[i] + [0]*(cardinality_U-tn_points_U[i]) for i in range(len(tU))])).type(torch.FloatTensor).view(tU.shape[0],cardinality_U,1)
tmask_V= torch.from_numpy(np.array([[1]*tn_points_V[i] + [0]*(cardinality_V-tn_points_V[i]) for i in range(len(tV))])).type(torch.FloatTensor).view(tV.shape[0],cardinality_V,1)

In [0]:
from tqdm import tqdm_notebook
import pickle
losses = []
test_losses = []
gold = torch.Tensor(labels).view(-1, 1)
tgold = torch.Tensor(tlabels).view(-1, 1)

n_epoch = 50
inputs = (U,V,  torch.from_numpy(np.array((list(map(int,n_points_U))))), torch.from_numpy(np.array(list(map(int,n_points_V)))),mask_U,mask_V)
t_input =(tU, tV, torch.from_numpy(np.array(list(map(int,tn_points_U)))), torch.from_numpy(np.array(list(map(int, tn_points_V)))),tmask_U,tmask_V)


In [0]:
type(cardinality_U)

In [0]:
# net=model_dict

In [0]:

aucs=[]

for _ in tqdm_notebook(range(n_epoch)):

    net.train()

    out = net(inputs)

    torch.save(net.state_dict, 'fs_pool_authors_keywords.model')

    loss = nn.BCEWithLogitsLoss()(out, gold)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    # print("after",_)
    losses.append(loss)
    net.eval()
    test_out = net(t_input)


    #aucs.append((roc_auc_score(gold,out),roc_auc_score(tgold,test_out)))


    test_loss = nn.BCEWithLogitsLoss()(test_out, tgold)
    # print("before",_)
    test_losses.append(test_loss)
    
    pickle.dump({'train_losses': losses, 'test_losses': test_losses}, open('fs_pool_authors_keywords.losses.pkl', 'wb'))

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type EMB_LAYER. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Embedding. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type BLP. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type FSEncoder. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/d

In [0]:
roc_auc_score(gold.cpu().detach().numpy(),nn.Sigmoid()(out).cpu().detach().numpy()),roc_auc_score(tgold.cpu().detach().numpy(),nn.Sigmoid()(test_out).cpu().detach().numpy())

In [0]:
import pickle
loss_dict = pickle.load(open('fs_pool_authors_keywords.losses.pkl', 'rb'))
model_dict = pickle.load(open('fs_pool_authors_keywords.model', 'rb'))

losses = loss_dict['train_losses']
test_losses = loss_dict['test_losses']

In [0]:
from matplotlib import pyplot as plt
n_epoch1 = len(losses)
plt.plot(range(n_epoch1), losses, label='train')
plt.plot(range(n_epoch1), test_losses, label='test')
plt.grid()
plt.legend()
plt.show()

In [0]:

# mse, cha, acc = torch.FloatTensor([-1, -1, -1])
# if not args.classify:
#     mse = (pred - points).pow(2).mean()
#     cha = chamfer_loss(pred, points)
#     if args.loss == 'direct':
#         loss = mse
#     elif args.loss == 'chamfer':
#         loss = cha
#     elif args.loss == 'hungarian':
#         loss = hungarian_loss(pred, points)
#     else:
#         raise NotImplementedError
# else:
#     loss = F.cross_entropy(pred, labels)
#     acc = (pred.max(dim=1)[1] == labels).float().mean()

# if train:
#     optimizer.zero_grad()
#     loss.backward()
#     optimizer.step()

# tracked_mse = tracker.update('{}_mse'.format(prefix), mse.item())
# tracked_cha = tracker.update('{}_cha'.format(prefix), cha.item())
# tracked_loss = tracker.update('{}_loss'.format(prefix), loss.item())
# tracked_acc = tracker.update('{}_acc'.format(prefix), acc.item())

# fmt = '{:.5f}'.format
# loader.set_postfix(
#     mse=fmt(tracked_mse),
#     cha=fmt(tracked_cha),
#     loss=fmt(tracked_loss),
#     acc=fmt(tracked_acc),
# )

# if args.show and not train:
#     #scatter(input_points, n_points, marker='o', transpose=args.mnist)
#     scatter(pred, n_points, marker='x', transpose=args.mnist)
#     plt.axes().set_aspect('equal', 'datalim')
#     plt.show()

In [0]:
# points, labels, n_points = samples[0]
# n_points

In [0]:
# import subprocess
# git_hash = subprocess.check_output(['git', 'rev-parse', 'HEAD'])

# torch.backends.cudnn.benchmark = True

# for epoch in range(args.epochs):
# tracker.new_epoch()
# with mp.Pool(4) as pool:
#     if not args.eval_only:
#         run(net, train_loader, optimizer, train=True, epoch=epoch, pool=pool)
#     if not args.train_only:
#         run(net, test_loader, optimizer, train=False, epoch=epoch, pool=pool)

# results = {
#     'name': args.name,
#     'tracker': tracker.data,
#     'weights': net.state_dict() if not args.multi_gpu else net.module.state_dict(),
#     'args': vars(args),
#     'hash': git_hash,
# }
# torch.save(results, os.path.join('logs', args.name))
# if args.eval_only:
#     break


In [0]:
# if __name__ == '__main__':
#     main()